<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D3_MongoDB_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Calcutta')))

Tested 2023-05-28 08:55:52.396738+05:30


##References
https://stackoverflow.com/questions/45142629/cant-connect-to-mongo-db-via-spark <br>
https://docs.mongodb.com/spark-connector/current/python-api/

#Packages

In [2]:
!pip3 install pymongo
from pymongo import MongoClient
import pandas as pd
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 21.9 MB/s eta 0:00:00


#Install & Test LOCAL MongoDB

In [3]:
!apt install mongodb > /dev/null
!service mongodb start



 * Starting database mongodb
   ...done.


In [4]:
!mongod -version

db version v3.6.8
git version: 8e540c0b6db93ce994cc548f000900bdc740f80a
OpenSSL version: OpenSSL 1.1.1f  31 Mar 2020
allocator: tcmalloc
modules: none
build environment:
    distarch: x86_64
    target_arch: x86_64


In [5]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

In [6]:
db = client.local
#db = client.sparkDB
#db = client.people
db.list_collection_names()

['startup_log']

In [7]:
table = db.startup_log
#table = db.coll01
#table = db.contacts
#table.count_documents({}) #gives the number of documents in the table
table.find_one()

{'_id': '1c7a782b5c18-1685244405616',
 'hostname': '1c7a782b5c18',
 'startTime': datetime.datetime(2023, 5, 28, 3, 26, 45),
 'startTimeLocal': 'Sun May 28 03:26:45.616',
 'cmdLine': {'config': '/etc/mongodb.conf',
  'net': {'bindIp': '127.0.0.1'},
  'storage': {'dbPath': '/var/lib/mongodb', 'journal': {'enabled': True}},
  'systemLog': {'destination': 'file',
   'logAppend': True,
   'path': '/var/log/mongodb/mongodb.log'}},
 'pid': 964,
 'buildinfo': {'version': '3.6.8',
  'gitVersion': '8e540c0b6db93ce994cc548f000900bdc740f80a',
  'modules': [],
  'allocator': 'tcmalloc',
  'javascriptEngine': 'mozjs',
  'sysInfo': 'deprecated',
  'versionArray': [3, 6, 8, 0],
  'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020',
   'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'},
  'buildEnvironment': {'distmod': '',
   'distarch': 'x86_64',
   'cc': 'cc: cc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0',
   'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknow

#Connect to and Test Remote MongoDB

#Install Spark

## Deprecated - dont use these commands

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark              --- no more required,  replaced by pyspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 


## Simple PIP3 install

In [8]:
!pip3 install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


#Spark2Mongo

In [9]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [ ]:
#sc.stop()

In [10]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

##Write to MongoDB

In [11]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [12]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [13]:
# this will send the data to the default database / collection specified during startup
people.write.format("mongo").mode("append").save()

In [14]:
# this will send the data to a new database / collection specified in this command
people.write.format("mongo").mode("append").option("database","people").option("collection", "contacts").save()

In [15]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

##Check with MongoDB client

In [16]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

In [17]:
db = client.people
list(db.list_collections())

[{'name': 'contacts',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'+\x90`\xa0Y\x13M\x19\xa9\xb8Wg\xe2\xa8f\xcc', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'people.contacts'}}]

In [18]:
db2 = client.sparkDB
list(db2.list_collections())

[{'name': 'coll01',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'5\x05e\xff\xddTL\x94\xaa\x80\xdb\xb12|\x96\x9f', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'sparkDB.coll01'}}]

In [19]:
kollection = db.contacts
c = kollection.find({},{"_id":0})
for crnt in c:
    print(crnt)

{'name': 'Dwalin', 'age': 169}
{'name': 'Oin', 'age': 167}
{'name': 'Gloin', 'age': 158}
{'name': 'Fili', 'age': 82}
{'name': 'Bombur'}
{'name': 'Bilbo Baggins', 'age': 50}
{'name': 'Gandalf', 'age': 1000}
{'name': 'Thorin', 'age': 195}
{'name': 'Balin', 'age': 178}
{'name': 'Kili', 'age': 77}


In [20]:
pandasDF = pd.DataFrame(kollection.find({},{"_id":0}))
pandasDF

,name,age
0,Dwalin,169.0
1,Oin,167.0
2,Gloin,158.0
3,Fili,82.0
4,Bombur,NaN
5,Bilbo Baggins,50.0
6,Gandalf,1000.0
7,Thorin,195.0
8,Balin,178.0
9,Kili,77.0


##Read from MongoDB

In [21]:
sparkDF = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/people.contacts").load()
sparkDF.show()

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|{6472cae14cdf8b2d...| 169|       Dwalin|
|{6472cae14cdf8b2d...| 167|          Oin|
|{6472cae14cdf8b2d...| 158|        Gloin|
|{6472cae14cdf8b2d...|  82|         Fili|
|{6472cae14cdf8b2d...|null|       Bombur|
|{6472cae14cdf8b2d...|  50|Bilbo Baggins|
|{6472cae14cdf8b2d...|1000|      Gandalf|
|{6472cae14cdf8b2d...| 195|       Thorin|
|{6472cae14cdf8b2d...| 178|        Balin|
|{6472cae14cdf8b2d...|  77|         Kili|
+--------------------+----+-------------+



In [22]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2023-05-28 09:01:18.800246+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)